### Setup

In [27]:
import os, sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='cobra')

from cobra import io
from scripts.helpers.model import add_single_gene_reaction_pair
from scripts.opt._fba import flux_balance_analysis

### Import Models

In [26]:
# Load models

# Load wildtype from manual directory (adjust path for notebooks directory)
# Use io.read_sbml_model for local files instead of io.load_model
wildtype = io.read_sbml_model('../data/manual/xmls/MNL_iCre1355_GAPFILL.xml')

models = {
    "Wildtype": wildtype
}

# Get altered reactions of wildtype - using correct directory name
# Available directories: KEGG_iBD1106_GAPFILL, KEGG_iCre1355_auto_GAPFILL, KEGG_iRC1080_GAPFILL, test
altered_dir = '../data/altered/xmls/MNL_iCre1355_GAPFILL/'

for root, dirs, files in os.walk(altered_dir):
    for file in files:
        if file.endswith('.xml'):
            # Get file name and remove .xml extension
            model_name = file[:-4]
            full_path = os.path.join(root, file)
            print(f"Loading model: {model_name} from {full_path}")
            models[model_name] = io.read_sbml_model(full_path)

# Print out loaded models
print(f"\nLoaded {len(models)} models:")
for name in models.keys():
    print(f"  - {name}: {models[name].name}")

No objective coefficients in model. Unclear what should be optimized


Loading model: SQE+MVA from ../data/altered/xmls/MNL_iCre1355_GAPFILL/SQE+MVA.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQE from ../data/altered/xmls/MNL_iCre1355_GAPFILL/SQE.xml


No objective coefficients in model. Unclear what should be optimized


...

Loading model: SQS+SQE from ../data/altered/xmls/MNL_iCre1355_GAPFILL/SQS+SQE.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQS from ../data/altered/xmls/MNL_iCre1355_GAPFILL/SQS.xml


No objective coefficients in model. Unclear what should be optimized



Loaded 7 models:
  - Wildtype: None
  - SQE+MVA: MNL_iCre1355_GAPFILL_SQE+MVA
  - SQE: MNL_iCre1355_GAPFILL_SQE
  - SQS+MVA: MNL_iCre1355_GAPFILL_SQS+MVA
  - SQS+SQE+MVA: MNL_iCre1355_GAPFILL_SQS+SQE+MVA
  - SQS+SQE: MNL_iCre1355_GAPFILL_SQS+SQE
  - SQS: MNL_iCre1355_GAPFILL_SQS


### Important IDs

In [ ]:
from enum import Enum

class MetID(Enum):
    # Pyruvate Compounds
    IPP = "ipdp_h"
    DMAPP = "dmpp_h"
    # Sterol Compounds
    ERG = "FILL_05"
    EPI = "FILL_02"
    METLOP = "FILL_01"
    FEC = "mfecostrl_c"
    EPOXY = "Ssq23epx_c"
    SQL = "sql_c"
    # MVA Compounds
    MVA1 = "C00356"
    MVA2 = "C00418"
    MVA3 = "C01107"
    MVA4 = "C01143"
    MVA5 = "C00009"

class RxnID(Enum):
    SS = 'SS'
    SS2 = 'ALT_SQS2'
    SQE = 'SMO'
    SQE2 = 'ALT_SQE2'
    CAS = 'CAS'
    ERG = 'RXNFILL_05'

### Preprocessing Models

In [25]:
for r in models['Wildtype'].reactions:
    if 'e' not in r.compartments: continue
    print(f"  - {r.id}: {r.name} | {r.reversibility} | {r.subsystem} | {r.metabolites}")

  - LIDOAMH: lidocaine amidohydrolase | False | Drug metabolism - cytochrome P450 | {<Metabolite lido_e at 0x13a24e10560>: -1.0, <Metabolite h2o_e at 0x13a24d63e30>: -1.0, <Metabolite 26dmani_e at 0x13a351fda30>: 1.0, <Metabolite degly_e at 0x13a24d44b30>: 1.0, <Metabolite h_e at 0x13a24d7e8d0>: 1.0}
  - MEGLYXYLAMH: monoethylglycinexylidide amidohydrolase | False | Drug metabolism - cytochrome P450 | {<Metabolite meglyxyl_e at 0x13a24d26000>: -1.0, <Metabolite h2o_e at 0x13a24d63e30>: -1.0, <Metabolite 26dmani_e at 0x13a351fda30>: 1.0, <Metabolite egly_e at 0x13a24d600e0>: 1.0, <Metabolite h_e at 0x13a24d7e8d0>: 1.0}
  - EX_h_e: H+ exchange | True | Exchange | {<Metabolite h_e at 0x13a24d7e8d0>: -1.0}
  - EX_h2o_e: H2O exchange | True | Exchange | {<Metabolite h2o_e at 0x13a24d63e30>: -1.0}
  - EX_pi_e: Phosphate exchange | True | Exchange | {<Metabolite pi_e at 0x13a24de7440>: -1.0}
  - EX_nh4_e: ammonia exchange | True | Exchange | {<Metabolite nh4_e at 0x13a24e011f0>: -1.0}
  - EX_

In [ ]:
for name in models.keys():
    model = models[name]

    add_single_gene_reaction_pair(
        model=model, 
        reaction_id="ERGOSTEROLEXCH", 
        metabolite_id="ergosterol_c", 
        description="Ergosterol exchange (assumption)", 
        reaction_type="Exchange", 
        gene_reaction_pairs=[(-1, MetID.ERG.value)]
        reversible=True
    )